### Load Data

In [36]:
import pandas as pd
import glob

# 1. 讀取 /mnt/data 資料夾下所有 CSV 檔案
csv_files = glob.glob("./result/*.csv")  # 如果路徑不同請自行調整
dfs = [pd.read_csv(path) for path in csv_files]

# 2. 合併為一個 DataFrame
df = pd.concat(dfs, ignore_index=True)


In [37]:
import pandas as pd
import glob

# 1. 讀取 /mnt/data 資料夾下所有 CSV 檔案並合併
csv_files = glob.glob("./result/*.csv")  # 如果路徑不同請自行調整
dfs = [pd.read_csv(path) for path in csv_files]
df = pd.concat(dfs, ignore_index=True)

# 2. 計算各 Model × Type 的平均分數
summary = (
    df
    .groupby(['Model', 'Type'])[['Exact Match Score', 'LLM Judge Score']]
    .mean()
    .reset_index()
)

# 3. 專注於 LLM Judge Score 的比較：pivot 並算差值（以兩個模型的順序判斷欄位）
judge_pivot = summary.pivot(index='Type', columns='Model', values='LLM Judge Score')
models = list(judge_pivot.columns)
judge_comparison = pd.DataFrame({
    'Type': judge_pivot.index,
    f'{models[0]}_Judge': judge_pivot.iloc[:,0],
    f'{models[1]}_Judge': judge_pivot.iloc[:,1],
    f'Δ_Judge ({models[1]} - {models[0]})': judge_pivot.iloc[:,1] - judge_pivot.iloc[:,0]
}).reset_index(drop=True)

# 4. 印出結果
print("=== 各 Model × Type 平均 Exact Match & LLM Judge Score ===")
print(summary.to_string(index=False))

print("\n=== LLM Judge Score 模型比較 ===")
print(judge_comparison.to_string(index=False))



=== 各 Model × Type 平均 Exact Match & LLM Judge Score ===
                 Model  Type  Exact Match Score  LLM Judge Score
         MiniCPM-V-2_6   圓餅圖           0.333333         2.666667
         MiniCPM-V-2_6 堆疊長條圖           0.076923         1.307692
         MiniCPM-V-2_6   折線圖           0.000000         1.000000
         MiniCPM-V-2_6   瀑布圖           0.000000         1.000000
         MiniCPM-V-2_6 群組長條圖           0.000000         2.000000
         MiniCPM-V-2_6    表格           0.000000         1.000000
         MiniCPM-V-2_6   長條圖           0.173913         1.695652
Qwen2.5-VL-7B-Instruct   圓餅圖           0.000000         4.333333
Qwen2.5-VL-7B-Instruct 堆疊長條圖           0.000000         2.538462
Qwen2.5-VL-7B-Instruct   折線圖           0.000000         3.000000
Qwen2.5-VL-7B-Instruct   瀑布圖           0.000000         4.000000
Qwen2.5-VL-7B-Instruct 群組長條圖           0.000000         1.666667
Qwen2.5-VL-7B-Instruct    表格           0.000000         1.000000
Qwen2.5-VL-7B-Instruct   長條圖      

In [38]:
# 假設 summary、judge_comparison 已經算好
summary.to_csv('./result/summary.csv', index=False, encoding='utf-8-sig')
judge_comparison.to_csv('./result/judge_comparison.csv', index=False, encoding='utf-8-sig')
print("已輸出 summary.csv 及 judge_comparison.csv 到 ./result/ 資料夾")


已輸出 summary.csv 及 judge_comparison.csv 到 ./result/ 資料夾


In [39]:
# -*- coding: utf-8 -*-
# 假设 summary、judge_comparison 已经算好

# 导出为 Excel
summary.to_excel('./result/summary.xlsx', index=False)
judge_comparison.to_excel('./result/judge_comparison.xlsx', index=False)

print("已输出 summary.xlsx 及 judge_comparison.xlsx 到 ./result/ 资料夹")


已输出 summary.xlsx 及 judge_comparison.xlsx 到 ./result/ 资料夹


In [40]:
# 1. 讀取 /mnt/data 資料夾下所有 CSV 檔案
csv_files = glob.glob("./result/*.csv")  # 如果路徑不同請自行調整
dfs = [pd.read_csv(path) for path in csv_files]

# 2. 合併為一個 DataFrame
df = pd.concat(dfs, ignore_index=True)


In [47]:
import pandas as pd
import glob
import os

# 1. Qwen
qwen_paths = glob.glob('./result/qwen_vlm_eval_with_llm_*.csv')
# 如果檔名其實是 ‘qwen_vlm_eval_with_llm__Locate.csv’ 兩個底線，請改成 '*.csv' 來通配

# 2. MiniCPM
minicpm_paths = glob.glob('./result/vlm_eval_with_llm_*.csv')

# 讀取與標註函式
def load_with_qtype(paths):
    dfs = []
    for path in paths:
        fname = os.path.basename(path).replace('.csv', '')
        qtype = fname.split('_')[-1].lower()
        df = pd.read_csv(path)
        df['question_type'] = qtype
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# 載入
df_qwen = load_with_qtype(qwen_paths)
df_minicpm = load_with_qtype(minicpm_paths)

print("Qwen DataFrame shape:", df_qwen.shape)
print("MiniCPM DataFrame shape:", df_minicpm.shape)


Qwen DataFrame shape: (53, 9)
MiniCPM DataFrame shape: (53, 9)


In [48]:
df_minicpm

,Type,Question,Short Answer,Long Answer,Prediction,Exact Match Score,LLM Judge Score,Model,question_type
0,群組長條圖,FY24 國泰產險的稅後淨利為多少億元？,2.6億,2.6億,無法從圖表中找到相關資訊,0,1,MiniCPM-V-2_6,extract
1,長條圖,FY24 的普通股每股淨值是多少？,54.3 元,54.3 元,根据提供的图片，FY24的普通股每股净值（Net Asset Value per Ordin...,0,1,MiniCPM-V-2_6,extract
2,長條圖,FY21海外獲利金額,為 9.1 億元,為 9.1 億元,找不到任何相關資訊,0,1,MiniCPM-V-2_6,extract
3,堆疊長條圖,請列出 FY24 手續費淨收益各構成項目的金額 單位：NT$B,信用卡 9.3、財富管理 16.0、外匯管理 0.01、聯貸 0.6、其他 1.9。,信用卡 9.3、財富管理 16.0、外匯管理 0.01、聯貸 0.6、其他 1.9。,根据图片中的数据图表，FY24手續費淨收益的構成項目包括：外匯保單債（31%）和外匯風險資產...,0,1,MiniCPM-V-2_6,extract
4,折線圖,FY20 的負債成本是多少？,3.83%,3.83%,在長條圖的標題部分，可以看到一個標題為'1Q25淨利差年增9bps'。在這個標題下方，有一個...,0,1,MiniCPM-V-2_6,extract
5,折線圖,FY24 的損益兩平資產報酬率為多少,2.83%,2.83%,找不到任何相關資訊,0,1,MiniCPM-V-2_6,extract
6,長條圖,請列出 1Q21～1Q25 的避險後投資收益率數值。,1Q21：6.31%\n\n1Q22：4.74%\n\n1Q23：2.69%\n\n1Q24...,1Q21：6.31%\n\n1Q22：4.74%\n\n1Q23：2.69%\n\n1Q24...,1Q24 至 FY24 的財富管理手續費淨收益數據如下：\n1. 財富管理手續費淨收益 (N...,0,1,MiniCPM-V-2_6,extract
7,圓餅圖,1Q25 的外幣資產避險成本為多少？,0.80%,0.80%,1Q25 的外幣資產避險成本為 0.80%。,1,5,MiniCPM-V-2_6,extract
8,群組長條圖,1Q25 的 25 個月續約率是多少？,95.50%,95.50%,根據圖片中的數據圖表，可以看到在 1Q25（第一季度第25個月）的續約率為 3.78%。,0,1,MiniCPM-V-2_6,extract
9,堆疊長條圖,FY24 的水險保費收入為多少？,收入為 1.5 NT$BN,收入為 1.5 NT$BN,"FY24/ FY23 水險保費收入為 430,672 萬美元。",0,1,MiniCPM-V-2_6,extract


In [49]:
import pandas as pd

# 1. miniCPM 平均分數
minicpm_summary = (
    df_minicpm
    .groupby('question_type')[['Exact Match Score', 'LLM Judge Score']]
    .mean()
    .reset_index()
    .rename(columns={
        'Exact Match Score': 'minicpm_EM_avg',
        'LLM Judge Score': 'minicpm_Judge_avg'
    })
)

# 2. Qwen 平均分數
qwen_summary = (
    df_qwen
    .groupby('question_type')[['Exact Match Score', 'LLM Judge Score']]
    .mean()
    .reset_index()
    .rename(columns={
        'Exact Match Score': 'qwen_EM_avg',
        'LLM Judge Score': 'qwen_Judge_avg'
    })
)

# 3. 合併比較
comparison = pd.merge(minicpm_summary, qwen_summary, on='question_type', how='outer')

# 4. 印出結果
print("=== miniCPM 平均分數 ===")
print(minicpm_summary.to_string(index=False))

print("\n=== Qwen 平均分數 ===")
print(qwen_summary.to_string(index=False))

print("\n=== 兩模型比較 ===")
print(comparison.to_string(index=False))


=== miniCPM 平均分數 ===
question_type  minicpm_EM_avg  minicpm_Judge_avg
      compare        0.200000           1.400000
      extract        0.083333           1.333333
       locate        0.000000           2.200000
         rank        0.000000           1.000000
        trend        0.300000           1.800000

=== Qwen 平均分數 ===
question_type  qwen_EM_avg  qwen_Judge_avg
      compare          0.0        3.300000
      extract          0.0        2.666667
       locate          0.0        3.300000
         rank          0.0        2.818182
        trend          0.0        3.400000

=== 兩模型比較 ===
question_type  minicpm_EM_avg  minicpm_Judge_avg  qwen_EM_avg  qwen_Judge_avg
      compare        0.200000           1.400000          0.0        3.300000
      extract        0.083333           1.333333          0.0        2.666667
       locate        0.000000           2.200000          0.0        3.300000
         rank        0.000000           1.000000          0.0        2.818182
   

In [50]:
import pandas as pd

# 1. miniCPM：按原始 Type 分組
minicpm_by_type = (
    df_minicpm
    .groupby('Type')[['Exact Match Score', 'LLM Judge Score']]
    .mean()
    .reset_index()
    .rename(columns={
        'Exact Match Score': 'minicpm_EM_avg',
        'LLM Judge Score' : 'minicpm_Judge_avg'
    })
)

# 2. Qwen：同樣按 Type
qwen_by_type = (
    df_qwen
    .groupby('Type')[['Exact Match Score', 'LLM Judge Score']]
    .mean()
    .reset_index()
    .rename(columns={
        'Exact Match Score': 'qwen_EM_avg',
        'LLM Judge Score' : 'qwen_Judge_avg'
    })
)

# 3. 合併兩張按 Type 分組的表
comparison_by_type = pd.merge(minicpm_by_type, qwen_by_type, on='Type', how='outer')

# 4. 印出結果
print("=== miniCPM 按 Type 的平均分數 ===")
print(minicpm_by_type.to_string(index=False))

print("\n=== Qwen 按 Type 的平均分數 ===")
print(qwen_by_type.to_string(index=False))

print("\n=== 依 Type 比較兩模型平均分數 ===")
print(comparison_by_type.to_string(index=False))



=== miniCPM 按 Type 的平均分數 ===
 Type  minicpm_EM_avg  minicpm_Judge_avg
  圓餅圖        0.333333           2.666667
堆疊長條圖        0.076923           1.307692
  折線圖        0.000000           1.000000
  瀑布圖        0.000000           1.000000
群組長條圖        0.000000           2.000000
   表格        0.000000           1.000000
  長條圖        0.173913           1.695652

=== Qwen 按 Type 的平均分數 ===
 Type  qwen_EM_avg  qwen_Judge_avg
  圓餅圖          0.0        4.333333
堆疊長條圖          0.0        2.538462
  折線圖          0.0        3.000000
  瀑布圖          0.0        4.000000
群組長條圖          0.0        1.666667
   表格          0.0        1.000000
  長條圖          0.0        3.434783

=== 依 Type 比較兩模型平均分數 ===
 Type  minicpm_EM_avg  minicpm_Judge_avg  qwen_EM_avg  qwen_Judge_avg
  圓餅圖        0.333333           2.666667          0.0        4.333333
堆疊長條圖        0.076923           1.307692          0.0        2.538462
  折線圖        0.000000           1.000000          0.0        3.000000
  瀑布圖        0.000000         